## BCH code

In [1]:
class BCHREPCode:
    
    def __init__(self, n, m, b, D):
        
        self.q = 2
        self.n = n
        self.m = m
        self.b = b
        self.D = D
            
        if (gcd(self.n, self.q) != 1):
            raise ValueError('Invalid input values: gcd(n,q) != 1.')
            
        if (self.n != self.q**self.m - 1):
            raise ValueError('Invalid input values: n != q^m - 1')
            
        
        # Initialize field
        self.BF = GF(self.q) # base field
        self.EF = GF(self.q**self.m) # extension field
        R.<x> = PolynomialRing(self.BF, 'x')
        self.R = R
        self.x = x
        self.R_RS = PolynomialRing(self.EF, 'X')
        self.alpha = self.EF.primitive_element()
        self.root_sequence = [self.alpha**i for i in range(self.b,self.b+self.D-1)]
        
        # define RS parameters
        self.k_RS = self.n - self.D + 1
        self.tau_RS = floor( (self.D - 1) / 2)
        self.alpha_RS = vector([self.alpha**i for i in range(self.n)])
        self.G_RS = matrix(self.EF, self.k_RS, self.n, lambda i,j : self.alpha_RS[j]**i)
        
        # Constructing generator matrix
        self.cosets = self.cyclotomic_cosets(self.n, self.q, self.b, self.D)
        
        self.generator_poly = self.BCH_generator_polynomial(self.x, self.alpha, self.D, self.cosets)
        
        if not (self.generator_poly.divides(self.x**self.n - 1)):
            raise ValueError('generator_poly is not a generator polynomial')
            
        self.k = self.n - self.generator_poly.degree()
        
        self.G = matrix(self.BF, self.k, self.n, lambda i,j : self.generator_poly[(j+(self.n-i)) % self.n])
        
        
    def cyclotomic_cosets(self, n, q, b, D):
        # compute cyclotomic cosets
    
        cosets = []

        for i in range(b,b+D-1):
            coset = [(i * q**j) % n for j in range(0,n-1)]
            coset = list(set(coset))
            coset.sort()
            cosets.append(coset)
        return cosets

    def minimal_polynomial(self, coset, x, alpha):
        # compute minimal polynomial from one coset
        poly = 1
        for j in range(len(coset)):
            poly *= (x - alpha**coset[j])
        return poly

    def BCH_generator_polynomial(self, x, alpha, D, cosets):
        # compute generator polynomial
        poly = self.minimal_polynomial(cosets[0], x, alpha)
        for i in range(1,D-1):
            poly = LCM(poly,self.minimal_polynomial(cosets[i],x,alpha))
        
        return poly
    
    
    def Encoding(self, message, zeropad = True):
        
        rem = len(message) % self.k
        
        if rem != 0:
            if zeropad:
                message.extend([self.BF(0)]*(self.k-rem))
            else:
                raise ValueError('k does not divide input size')
                
                
        c = []
        
        # Encoding each chunk of size k
        for i in range(0, len(message), self.k):
            c.extend(self.EncodeChunkBCH(message[i:i+self.k]))
        
        return c
            
    def EncodeChunkBCH(self, chunk):
        
        # Encode a chunk of size k
        if len(chunk) != self.k:
            raise ValueError('Invalid chunk size')
            
        c = vector(self.BF, chunk) * self.G
        
        c_BCH_REP = self.EncodeChunkREP(c)
        
        return c_BCH_REP
    
    def EncodeChunkREP(self, chunk):
        
        c_BCH_REP = []
        
        for i in range(len(chunk)):
            for j in range(8): # BCH length n_BCH
                c_BCH_REP.append(chunk[i])
                
        return c_BCH_REP
                
    
    def Decoding(self, r):
        
        # Check input size
        if len(r) % self.n != 0:
            raise ValueError('Invalid input size')
            
        c = []
        
        for i in range(0,len(r),8): # BCH length n_BCH
            c.append(self.DecodeChunkREP(r[i:i+8]))
        
        
        for i in range(len(c)):
            c[i] = self.EF(c[i])
            
        c_final = []
        
        for i in range(0,len(c),self.n):
            c_final.extend(self.BivariateInterpolation(c[i:i+self.n]))
            
        c_final = self.EncodingRS(c_final)
        
        c_final = self.DecodingBCH(c_final)
        
        return c_final
    
    
    def DecodingBCH(self, c):
        
        if len(c) % self.n != 0:
            raise ValueError('Invalid input size')
            
        for i in range(len(c)):
            c[i] = self.BF(c[i])
            
        c_final = []
        
        for i in range(0,len(c),self.n):
            c_final.extend(self.DecodeChunkBCH(c[i:i+self.n]))
            
        return c_final
    
    def DecodeChunkBCH(self, chunk):
        
        cols = self.G.pivots()
        G_independent = self.G.matrix_from_columns(cols)
        chunk_independent = [chunk[i] for i in cols]
        
        return vector(self.BF, chunk_independent) * G_independent.inverse()
    
    
    def EncodingRS(self, m, zeropad = True):
        
        rem = len(m) % self.k_RS
        
        if rem != 0:
            if zeropad:
                m.extend([self.EF(0)]*(self.k_RS-rem))
            else:
                raise ValueError('k does not divide input size')
                
                
        c = []
        
        # Encoding each chunk of size k
        for i in range(0, len(m), self.k_RS):
            c.extend(self.EncodeChunkRS(m[i:i+self.k_RS]))
        
        return c
            
    def EncodeChunkRS(self, chunk):
        
        # Encode a chunk of size k
        if len(chunk) != self.k_RS:
            raise ValueError('Invalid chunk size')
            
        c = vector(self.EF, chunk) * self.G_RS
        return c
    
    def BivariateInterpolation(self, chunk):
        
        if len(chunk) != self.n:
            raise ValueError('Invalid chunk size')
            
        # Constructing matrices
        M1 = matrix(self.EF, self.n, self.tau_RS + self.k_RS, lambda i,j : self.alpha_RS[i]**j)
        M2 = matrix(self.EF, self.n, self.tau_RS + 1, lambda i,j : chunk[i] * self.alpha_RS[i]**j)
        M = M1.augment(M2)
        
        # Solving system
        RK = M.right_kernel()
        
        if len(RK.basis()) == 0:
            return(None)
        
        sol = RK.basis()[0]

        # Constructing Q0 and Q1 polynomials
        Q0 = self.R_RS(list(sol[:self.tau_RS+self.k_RS]))
        Q1 = self.R_RS(list(sol[self.tau_RS+self.k_RS:]))

        # Calculating -Q0/Q1
        q, r = Q0.quo_rem(Q1)

        if r != 0:
            #print('Non-zero remainder (possibly >tau errors). Returning None')
            return(None)

        out = []

        out.extend((-q).list())
        out.extend([self.EF(0)]*(self.k_RS-len(out)))

        return out
    
    def DecodeChunkREP(self, chunk):
        if chunk.count(1) >= floor(8 / 2): # BCH length n_BCH
            return 1
        elif chunk.count(1) < floor(8 / 2):
            return 0

In [2]:
C = BCHREPCode(n = 15, m = 4, b = 1, D = 7)

In [3]:
message = [1,1,0,1,1]
c = C.Encoding(message)
print('Codeword:', c)
d = C.Decoding(c)
print('Decoded word:', d)

Codeword: [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Decoded word: [1, 1, 0, 1, 1]
